<a href="https://colab.research.google.com/github/manthojunagachaitanya/Generative_AI/blob/main/GAI__lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model

# Step 1: Load and preprocess the dataset
url = "Housing.csv"  # Update this with your dataset's path
dataset = pd.read_csv(url)

# Assuming 'price' is the target column and the rest are features
X = dataset.drop('price', axis=1)  # Features
y = dataset['price']  # Target variable

# ----> Identify and handle categorical columns (e.g., using OneHotEncoder) <----
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']  # Replace with your categorical column names

# Create a OneHotEncoder instance (we set handle_unknown='ignore' to avoid errors for unseen categories)
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the categorical features
encoded_cols = encoder.fit_transform(X[categorical_cols])

# Create a DataFrame from the encoded features
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))

# Drop the original categorical columns and concatenate the encoded features
X = X.drop(categorical_cols, axis=1)
X = pd.concat([X, encoded_df], axis=1)

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 4: Build the ANN model
model = Sequential()

# Hidden layers based on the provided architecture
model.add(Dense(15, input_dim=X_train.shape[1], activation='relu'))  # Hidden Layer 1
model.add(Dense(20, activation='relu'))  # Hidden Layer 2
model.add(Dense(25, activation='relu'))  # Hidden Layer 3
model.add(Dense(20, activation='relu'))  # Hidden Layer 4
model.add(Dense(15, activation='relu'))  # Hidden Layer 5

# Output layer (since we're predicting a continuous value, we use linear activation)
model.add(Dense(1, activation='linear'))

# Step 5: Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Step 6: Train the model
history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))

# Step 7: Evaluate the model
train_loss, train_mae = model.evaluate(X_train, y_train)
test_loss, test_mae = model.evaluate(X_test, y_test)

print(f"Training MAE: {train_mae}")
print(f"Testing MAE: {test_mae}")

# Step 8: Save the model in .h5 format
model.save('housing_price_predictor.h5')

# Step 9: Load the model for deployment
loaded_model = load_model('housing_price_predictor.h5')

# Step 10: Make predictions with the loaded model
predictions = loaded_model.predict(X_test)

# Display predictions vs actual values for the first 10 examples
for i in range(10):
    print(f"Predicted: {predictions[i][0]}, Actual: {y_test.iloc[i]}")